[Node 57: SAX Parser](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node57.html)

Navigation:

**Next:** [JSON – Lightweight Alternative](node58.ipynb) **Up:** [Datenformate II – XML](node53.ipynb) **Previous:** [XML Struktur im Detail](node56.ipynb)

##  SAX Parser
 
SAX ist ein weitverbreiteter XML-Parser. Es gibt Versionen für Perl, Python, Java, ...

SAX verarbeitet Dokumente als   <font color=#008000> *streams*</font>, d.h. das Dokument wird abschnittsweise verarbeitet.  

Alternativer gängiger Parser ist DOM (Document Object Model), der ganzes Dokument liest und intern  im Speicher bereitstellt $\Rightarrow$ gut geeignet für komplexe baumartige XML-Strukturen.


SAX ist gutes Beispiel für Anwendung von objekt-orientiertem Programmieren: nutzt Vererbung und Überschreiben von Methoden. 

Verwendung:  
*  <font color=#008000> *Handler Klasse*</font>  definieren $\Rightarrow$ erbt von   <font color=#0000e6> ``SAX ContentHandler``</font> 
* Vorgegebene Methoden der Klasse bei Bedarf implementieren:   <font color=#0000e6> ``startElement, endElement, ...``</font> 
* Diese werden beim "Document–Parsing"   <font color=#008000> *automatisch*</font>  aufgerufen (ähnlich wie call-back Funktionen in  TkInter-GUI) 
* Die Handler Klasser wird SAX übergeben vor dem Document–Parsing  

### XML Beispiel (article.xml)
```xml
<?xml version="1.0"?>
<webArticle category="news" subcategory="technical">
    <header title="NASA Builds Warp Drive"
           length="3k"
           author="Joe Reporter"
           distribution="all"/>
    <body>Seattle, WA - Today an anonymous individual
           announced that NASA has completed building a
           Warp Drive and has parked a ship which uses
           the drive in his back yard.  This individual
           claims that although he hasn't been contacted by
           NASA concerning the parked space vessel, he assumes
           that he will be launching it later this week to
           mount an exhibition to the Andromeda Galaxy.
    </body>
</webArticle>
```


Handler–KlasseXML  (simplehandler.py):  


In [ ]:
from xml.sax.handler import ContentHandler

class ArticleHandler(ContentHandler):
  """A handler to deal with articles in XML"""

  def startElement(self, name, attrs):
    print ("Start element:",name)

  def endElement(self, name):
    print ("End element:",name)

def main():
  from xml.sax  import make_parser
  
  ch = ArticleHandler()
  saxparser = make_parser()
  
  saxparser.setContentHandler(ch)
  saxparser.parse("data/xml_article.xml")

if __name__ == '__main__':
  main()


### Etwas mehr XML processing

* Handler erweitern und tag-spezifisches Prozessieren einbauen 
* zusätzliche Methode   <font color=#0000e6> ``characters()``</font>  zum Prozessieren der eigentlichen Daten



In [ ]:
from xml.sax.handler import ContentHandler

class ArticleHandler(ContentHandler):
  """A handler to deal with articles in XML"""

  inArticle = 0
  inBody    = 0
  isMatch   = 0
  title     = ""
  body      = ""

  def startElement(self, name, attrs):
    if name == "webArticle":
      subcat = attrs.get("subcategory", "")
      if subcat.find("tech") > -1:
        self.inArticle = 1
        self.isMatch = 1

    elif self.inArticle:
      if name == "header":
        self.title = attrs.get("title", "")
      if name == "body":
        self.inBody = 1

  def characters(self, characters):
    MAXLEN=800
    if self.inBody:
      if len(self.body) < MAXLEN:
        self.body += characters
      if len(self.body) > MAXLEN:
        self.body = self.body[:MAXLEN-2] + "..."
        self.inBody = 0

  def endElement(self, name):
    if name == "body":
      self.inBody = 0

def main():
  import sys
  
  from xml.sax  import make_parser
  
  ch = ArticleHandler()
  saxparser = make_parser()
  
  saxparser.setContentHandler(ch)
  saxparser.parse("data/xml_article.xml")

  if ch.isMatch:
      print ("News Item!")
      print ("Title:", ch.title)
      print ("Body:", ch.body)

if __name__ == '__main__':
  main()


## Elementtree Parser
Eine weitere einfache Alternative für simple XML-Dokumente ist der    <font color=#0000ff> **Elementtree**</font> -Parser, siehe Dokumentation / Tutorial in https://docs.python.org/3/library/xml.etree.elementtree.html 

Im einfachsten Fall wird das gesamte XML Dokument eingelesen, die einzelnen Elemente sind als **verschachtelte Iterables**  zugänglich

In [ ]:
# angewandt auf obiges Beispiel
import xml.etree.ElementTree as ET
tree = ET.parse('data/xml_article.xml')
root = tree.getroot()
print(root)
print(root.attrib)
print(root[0])
for el in root:
    print ('Element:',el.tag)
    if el.tag =='body':
        print(el.text[:200])